# Exploring Repressor Titration Across Carbon Sources

In [3]:
import sys
sys.path.insert(0, '../../')
import numpy as np
import pandas as pd
import bokeh.io
import bokeh.plotting
import matplotlib.pyplot as plt
import mwc.viz
import mwc.model
mwc.viz.set_style()
constants = mwc.model.load_constants()
bokeh.io.output_notebook()

Loading BokehJS ...

## Loading of Data

In [26]:
fc_data = pd.read_csv('../../data/counted_repressors.csv')
fluct_data = pd.read_csv('../../data/compiled_fluctuations.csv')
carbon = ['glucose', 'acetate', 'glycerol']
pars = ['carbon', 'alpha_1', 'alpha_2']
samples = []
for c in carbon:
    _samples = pd.read_csv(f'../../data/{c}_calibration_factor_samples.csv')
    columns = [k for k in _samples.keys() for p in pars if p in k and 'log' not in k and 'raw' not in k]
    _samples = _samples[columns]
    _samples['carbon'] = c
    samples.append(_samples)        
samples = pd.concat(samples, sort=False)

## Showing variability in the calibration factor

## Exploring repressor distributions and determining noise floor

In [27]:
colors = bokeh.palettes.Category10_10

In [44]:
p = bokeh.plotting.figure(width=300, height=300, 
                         x_axis_label='repressors per cell',
                         y_axis_label='frequency')
p2 = bokeh.plotting.figure(width=300, height=300,
                          x_axis_label='repressors per cell',
                          y_axis_label='ECDF')
color_key = {c:colors[i] for i, c in enumerate(fc_data['carbon'].unique())}
mean = {}
mean_p_5sigma = {}
for g, d in fc_data.groupby('carbon'):
    d = d[d['strain']=='auto'].copy()
    y = np.arange(0, len(d), 1) / len(d)
    x_mode = np.sort(d['repressors'])
    x_min = np.sort(d['repressors_min'])
    x_max = np.sort(d['repressors_max'])
    mean[g] = np.mean(x_mode)
    mean_p_5sigma[g] = np.percentile(x_mode, 97.5)
    hist, bins = np.histogram(d['repressors'], bins=1000, density=True)
    p.step(bins[:-1], hist, color=color_key[g], line_width=1)
    p2.line(x_mode, y, color=color_key[g], line_width=1)

layout = bokeh.layouts.Row(p, p2)
bokeh.io.show(layout)

In [79]:
# Apply the 97.5_threshold for each
fc_thresh = fc_data[fc_data['repressors'] >= 20]
fc_day_data = pd.read_csv('../../data/compiled_fold_change.csv')
fc_day_thresh = fc_day_data[fc_day_data['repressors'] >= 20]

In [80]:
p = bokeh.plotting.figure(width=300, height=300, 
                         x_axis_label='repressors per cell',
                         y_axis_label='frequency')
p2 = bokeh.plotting.figure(width=300, height=300,
                          x_axis_label='repressors per cell',
                          y_axis_label='ECDF')
color_key = {c:colors[i] for i, c in enumerate(fc_data['carbon'].unique())}
mean = {}
mean_p_5sigma = {}
for g, d in fc_day_data.groupby('carbon'):
    d = d[d['strain']=='auto'].copy()
    y = np.arange(0, len(d), 1) / len(d)
    x_mode = np.sort(d['repressors'])
    x_min = np.sort(d['repressors_min'])
    x_max = np.sort(d['repressors_max'])
    mean[g] = np.mean(x_mode)
    mean_p_5sigma[g] = np.percentile(x_mode, 97.5)
#     hist, bins = np.histogram(d['repressors'], bins=1000, density=True)
    p.step(bins[:-1], hist, color=color_key[g], line_width=1)
    p2.line(x_mode, y, color=color_key[g], line_width=1)

layout = bokeh.layouts.Row(p, p2)
bokeh.io.show(layout)

In [90]:
p = bokeh.plotting.figure(width=400, height=300, 
                         x_axis_label='atc [ng / mL]',
                         y_axis_label='repressors per cell',
                         y_axis_type='log')
p2 = bokeh.plotting.figure(width=400, height=300, 
                         x_axis_label='repressors per cell',
                         y_axis_label='fold-change',
                         y_axis_type='log',
                         x_axis_type='log')

rep_range = np.logspace(0, 4, 300)
arch = mwc.model.SimpleRepression(rep_range, ep_r=constants['O2'],
                                 ka=constants['ka'], ki=constants['ki'],
                                 effector_conc=0, ep_ai=4.5).fold_change()
p2.line(rep_range, arch, color='black', line_width=1.5)
for g, d in fc_day_data[fc_day_data['strain']=='dilution'].groupby(['carbon']):
    p2.circle(d['repressors'], d['fold_change'], color=color_key[g], size=0.01)
#     _grouped = d.groupby(['atc_ngml'])[['repressors', 'fold_change']].mean().reset_index()
#     _grouped2 = d.groupby(['atc_ngml'])[['repressors', 'fold_change']].mean()
#     p.circle(_grouped['atc_ngml'], _grouped['repressors'], legend=g, color=color_key[g])
#     p.line(_grouped['atc_ngml'], _grouped['repressors'], legend=g, color=color_key[g])
#     p2.circle(_grouped2['repressors'],_grouped2['fold_change'], legend=g, color=color_key[g])
p.legend.location = 'bottom_right'
p.legend.location = 'bottom_left'
layout = bokeh.layouts.Row(p, p2)
bokeh.io.show(layout)

In [67]:
fc_data

,strain,area_pix,mean_yfp,mean_mCherry,atc_ngml,date,carbon,temp,operator,run_number,yfp_bg_val,mCherry_bg_val,alpha_mode,alpha_hpd_min,alpha_hpd_max,fold_change,total_mCherry,repressors,repressors_min,repressors_max
0,delta,386.0,954.991116,-4.203771,0.0,20181019,acetate,37,O2,1,156.540946,186.407486,848.699003,702.729821,1019.711171,0.927590,-1622.655735,-2.037899,-2.164327,-1.917217
1,delta,535.0,1037.419187,-2.111359,0.0,20181019,acetate,37,O2,1,156.540946,186.407486,848.699003,702.729821,1019.711171,1.007653,-1129.577249,-1.418640,-1.506650,-1.334630
2,delta,406.0,769.810509,0.643698,0.0,20181019,acetate,37,O2,1,156.540946,186.407486,848.699003,702.729821,1019.711171,0.747723,261.341378,0.328219,0.308783,0.348582
3,delta,449.0,901.619449,-3.356269,0.0,20181019,acetate,37,O2,1,156.540946,186.407486,848.699003,702.729821,1019.711171,0.875750,-1506.964831,-1.892602,-2.010016,-1.780525
4,delta,450.0,993.558128,6.662078,0.0,20181019,acetate,37,O2,1,156.540946,186.407486,848.699003,702.729821,1019.711171,0.965051,2997.935024,3.765116,3.542152,3.998698
5,delta,508.0,944.318102,4.183320,0.0,20181019,acetate,37,O2,1,156.540946,186.407486,848.699003,702.729821,1019.711171,0.917223,2125.126650,2.668954,2.510902,2.834531
6,delta,489.0,1238.701686,-2.427343,0.0,20181019,acetate,37,O2,1,156.540946,186.407486,848.699003,702.729821,1019.711171,1.203160,-1186.970607,-1.490720,-1.583202,-1.402442
7,delta,415.0,813.764554,1.957687,0.0,20181019,acetate,37,O2,1,156.540946,186.407486,848.699003,702.729821,1019.711171,0.790416,812.440078,1.020346,0.959923,1.083647
8,delta,342.0,1101.875672,0.472955,0.0,20181019,acetate,37,O2,1,156.540946,186.407486,848.699003,702.729821,1019.711171,1.070260,161.750619,0.203143,0.191113,0.215746
9,delta,432.0,1590.936832,10.758652,0.0,20181019,acetate,37,O2,1,156.540946,186.407486,848.699003,702.729821,1019.711171,1.545289,4647.737623,5.837109,5.491444,6.199234
